# 02 — Geocoding

In [17]:
# Imports & paths
import os, json, time, hashlib, re
import pandas as pd
import requests
from dotenv import load_dotenv
from pathlib import Path

def find_project_root(start: Path, marker_dir: str = "data") -> Path:
    cur = start.resolve()
    # if we're in a file context, go to its parent; if in a dir (Jupyter), keep as is
    if cur.is_file():
        cur = cur.parent
    # Walk up until we find a directory containing marker_dir (e.g., 'data')
    while cur != cur.parent:
        if (cur / marker_dir).exists():
            return cur
        cur = cur.parent
    # Fallback: original start dir
    return start.resolve()

# Works both in Jupyter (no __file__) and when run as a script
try:
    _start = Path(__file__)
except NameError:
    _start = Path.cwd()

PROJECT_ROOT = find_project_root(_start, marker_dir="data")
DATA_DIR = PROJECT_ROOT / "data"
RAW_DIR = DATA_DIR / "raw"
INTERIM_DIR = DATA_DIR / "interim"
PROCESSED_DIR = DATA_DIR / "processed"

for d in [RAW_DIR, INTERIM_DIR, PROCESSED_DIR]:
    d.mkdir(parents=True, exist_ok=True)

print("Project root:", PROJECT_ROOT)
print("Interim dir:", INTERIM_DIR)

Project root: C:\Users\zhant\toronto-rentals
Interim dir: C:\Users\zhant\toronto-rentals\data\interim


In [18]:
# Load API key from .env
from dotenv import load_dotenv
load_dotenv(dotenv_path=PROJECT_ROOT / '.env')
import os
API_KEY = os.getenv('GOOGLE_MAPS_API_KEY')
assert API_KEY and API_KEY.strip(), 'GOOGLE_MAPS_API_KEY not set in .env'

## 1) Load cleaned listings

In [19]:
clean_path = INTERIM_DIR / 'clean_listings.csv'
assert clean_path.exists(), f'File not found: {clean_path}'

df = pd.read_csv(clean_path)
print(df.shape)
df.head(3)

(89, 14)


,title,price,sqft,bedrooms,bathrooms,address,neighbourhood,url,source,url_canon,address_norm,unit,source_rank,variant_id
0,NaN,2995.0,750.0,1.5,1.0,"35 Mariner Terr - Toronto, ON",NaN,NaN,NaN,NaN,35 mariner terr - toronto on,NaN,999,69fbb2161b
1,NaN,2800.0,600.0,1.0,1.0,"224 King Street West - Toronto, ON",NaN,NaN,NaN,NaN,224 king street west - toronto on,NaN,999,7140165d6e
2,NaN,2095.0,500.0,1.0,1.0,"11 Brunel Court - Toronto, ON",NaN,NaN,NaN,NaN,11 brunel court - toronto on,NaN,999,fd258ce795


In [20]:
CACHE_DIR = INTERIM_DIR / 'geocode_cache'
CACHE_DIR.mkdir(parents=True, exist_ok=True)

GEOCODE_URL = 'https://maps.googleapis.com/maps/api/geocode/json'

def normalize_address(addr: str) -> str:
    if pd.isna(addr):
        return ''
    s = str(addr).strip()
    s = re.sub(r'\s+', ' ', s)
    return s

def cache_key(addr: str) -> Path:
    h = hashlib.sha1(addr.lower().encode()).hexdigest()
    return CACHE_DIR / f'{h}.json'

def geocode(address: str, api_key: str, max_retries: int = 5, base_sleep: float = 0.5):
    """Return (lat, lon, status, resolved_address) or (None, None, status, None)"""
    addr = normalize_address(address)
    if not addr:
        return None, None, 'EMPTY_ADDRESS', None

    # Heuristic: ensure city/province for better accuracy
    addr_q = addr
    if 'toronto' not in addr.lower():
        addr_q += ', Toronto'
    if 'on' not in addr.lower() and 'ontario' not in addr.lower():
        addr_q += ', ON'
    addr_q += ', Canada'

    # Cache
    cpath = cache_key(addr_q)
    if cpath.exists():
        data = json.loads(cpath.read_text(encoding='utf-8'))
        return data['lat'], data['lon'], data.get('status','CACHED_OK'), data.get('formatted_address')

    params = {
        'address': addr_q,
        'key': api_key,
        'components': 'country:CA|administrative_area:ON|locality:Toronto'
    }

    for attempt in range(max_retries):
        r = requests.get(GEOCODE_URL, params=params, timeout=20)
        data = r.json()
        status = data.get('status', '')
        if status == 'OK' and data.get('results'):
            loc = data['results'][0]['geometry']['location']
            fmt = data['results'][0].get('formatted_address')
            cpath.write_text(json.dumps({'lat': loc['lat'], 'lon': loc['lng'], 'status': status, 'formatted_address': fmt}), encoding='utf-8')
            return loc['lat'], loc['lng'], status, fmt
        elif status in {'OVER_QUERY_LIMIT','RESOURCE_EXHAUSTED'}:
            sleep = base_sleep * (2 ** attempt)
            time.sleep(sleep)
            continue
        else:
            # ZERO_RESULTS / REQUEST_DENIED / INVALID_REQUEST / ...
            cpath.write_text(json.dumps({'lat': None, 'lon': None, 'status': status, 'formatted_address': None}), encoding='utf-8')
            return None, None, status, None

    return None, None, 'RETRY_EXCEEDED', None

In [21]:
# Prepare columns
for col in ['lat','lon','geocode_status','formatted_address']:
    if col not in df.columns:
        df[col] = pd.NA

mask = (df['lat'].isna() | df['lon'].isna()) & df['address'].notna() & (df['address'].str.strip() != '')

pending = df[mask]
print('Rows to geocode:', len(pending))

fail_log = []

for idx, row in pending.iterrows():
    lat, lon, status, fmt = geocode(row['address'], API_KEY)
    df.at[idx, 'lat'] = lat
    df.at[idx, 'lon'] = lon
    df.at[idx, 'geocode_status'] = status
    df.at[idx, 'formatted_address'] = fmt

    if status not in ('OK','CACHED_OK'):
        fail_log.append({'index': int(idx), 'address': row['address'], 'status': status})

    # polite delay to avoid hitting per-second limits
    time.sleep(0.15)

print('Done. Failures:', len(fail_log))

Rows to geocode: 89
Done. Failures: 0


In [22]:
out_path = INTERIM_DIR / 'rentals_with_coords.csv'
df.to_csv(out_path, index=False)
print('Saved:', out_path.resolve())

if len(fail_log):
    fail_path = INTERIM_DIR / 'geocode_failures.csv'
    pd.DataFrame(fail_log).to_csv(fail_path, index=False)
    print('Failures log:', fail_path.resolve())

cols = [c for c in ['address','formatted_address','lat','lon','geocode_status','price','sqft','bedrooms','bathrooms'] if c in df.columns]
df[cols].head(10)

Saved: C:\Users\zhant\toronto-rentals\data\interim\rentals_with_coords.csv


,address,formatted_address,lat,lon,geocode_status,price,sqft,bedrooms,bathrooms
0,"35 Mariner Terr - Toronto, ON","35 Mariner Terrace, Toronto, ON M5V 3V9, Canada",43.639843,-79.392096,OK,2995.0,750.0,1.5,1.0
1,"224 King Street West - Toronto, ON","2906-224 King St W, Toronto, ON M5H 0A6, Canada",43.64753,-79.387334,OK,2800.0,600.0,1.0,1.0
2,"11 Brunel Court - Toronto, ON","11 Brunel Ct, Toronto, ON M5V 3Y3, Canada",43.639368,-79.393315,OK,2095.0,500.0,1.0,1.0
3,"85 Queens Wharf Road - Toronto, ON","85 Queens Wharf Rd, Toronto, ON M5V 0J9, Canada",43.638942,-79.398814,OK,4200.0,1200.0,2.0,2.0
4,"81 Navy Wharf Court - Toronto, ON","81 Navy Wharf Ct, Toronto, ON M5V 3M5, Canada",43.64122,-79.391425,OK,2300.0,600.0,1.0,1.0
5,"333 Adelaide Street East - Toronto, ON","333A Adelaide St E, Toronto, ON M5A 1N2, Canada",43.651878,-79.369249,OK,2925.0,750.0,2.0,1.0
6,"488 University Avenue - Toronto, ON","488 University Ave, Toronto, ON M5G 0C1, Canada",43.655092,-79.388993,OK,3250.0,799.0,2.0,2.0
7,"832 Bay Street - Toronto, ON","Hamghlin Building, 832 Bay St., Toronto, ON M5...",43.662104,-79.386591,OK,2400.0,599.0,1.0,1.0
8,"77 & 99 Gerrard Street West - Toronto, ON","Gerrard St W, Toronto, ON, Canada",43.658317,-79.385667,OK,2250.0,525.0,1.0,1.0
9,"131 Bloor Street West - Toronto, ON","131 Bloor St W, Toronto, ON M5S 3L7, Canada",43.668796,-79.392329,OK,3099.0,820.0,1.0,1.0
